In [1]:
from CARP_solver import *
from cProfile import run

In [2]:
info = read_file("../datasets/egl-s1-A.dat")
gv.init(info)

In [3]:
def hamming_distance(s1: Solution, s2: Solution):
    def deadhead(solu: Solution):
        deadhead_link = []
        for route in solu.routes:
            for idx in range(len(route.tasks) - 1):
                deadhead_link.append((route[idx].t, route[idx + 1].s))

        return deadhead_link

    m = min(len(s1.routes), len(s2.routes))
    n = gv.num_tasks
    common = len(set(deadhead(s1)).intersection(set(deadhead(s2))))
    return n + m - common


def pop_dist_rank(pop):
    """计算每个solu在pop中的平均距离，倒序排序，越大排名越高"""
    pop_size = len(pop)  # ! 最后要减一
    dist_matrix = np.zeros((pop_size, pop_size), dtype=int)
    for i in range(pop_size):
        for j in range(i + 1, pop_size):
            dist = hamming_distance(pop[i], pop[j])
            dist_matrix[i, j] = dist_matrix[j, i] = dist
    
    # 添加随机值[0,1)进行排序
    return np.argsort(-dist_matrix.sum(axis=0))


def QDNS_update_pop(pop, solu):
    if not solu.feasible():
        return False

    pop_size = len(pop)
    pop.append(solu)

    cost_rank = np.argsort(np.array([solu.cost for solu in pop]) + np.random.rand(pop_size + 1))
    dist_rank = pop_dist_rank(pop)
    alpha = 0.6
    QDF = [alpha * cost_rank[i] + (1 - alpha) * dist_rank[i] for i in range(pop_size + 1)]
    # 将要被抛弃的pop的idx
    worst = np.argmax(QDF)
    # print(list(map(lambda x: x.cost, pop)))
    # print(QDF, worst)
    pop.pop(worst)  # 直接在原来的pop上改
    if worst == pop_size:
        # 弹出了新加的solu
        return False

    return True
    
    

In [4]:
def main(pop_size, timeout):
    pop = init_pop(pop_size)
    best_solu = min(pop, key=lambda x: x.cost)
    tabu_list = []
    max_len_tabu = 30

    start = end = perf_counter()
    while end - start < timeout:
        # crossover
        co = sample(pop, k=2)
        offspring = Solution.crossover(co[0], co[1])

        # 生成一个随机数，判断应该进行怎样improve
        ty = randint(0, 2)
        if ty == 0:
            pass
        elif ty == 1:
            # single insertion
            offspring = single_insert(offspring, best_solu.cost, tabu_list, 10)
            pass
        
        elif ty == 2:
            # merge split
            # tmp = offspring.
            # for i in range(20):
            #     new_offspring = merge_split(offspring)
            #     if new_offspring.cost < best_offspring.cost:
            #         best_offspring = new_offspring
            
            # offspring = best_offspring
            offspring = merge_split(offspring)
            
        
        if offspring is not None and offspring not in pop and offspring not in tabu_list:
            # print(offspring.feasible(), offspring.cost)

            # QDNS_update_pop(pop, offspring)
            update_pop(pop, offspring)
            # new_solu.assert_demand()
        
            tabu_list.append(offspring)

        bfs = best_feasible_solu(pop)
        # print(pop)
        
        
        if bfs.cost < best_solu.cost:
            best_solu = bfs.dcopy()
            print(sorted(list(map(lambda x: x.cost, pop))))
            print(best_solu)
            

        # 删除禁忌表中留存时间过长的解
        if len(tabu_list) >= max_len_tabu:
            tabu_list.pop(0)

        end = perf_counter()
    
    return best_solu, pop

In [5]:
# run("print(main(20, 5))")
best, pop = main(10, 100)

[15162, 16493, 16803, 16903, 16919, 17067, 17243, 17443, 17596, 18031, 18538]
solution cost:15162, feasible
{'tasks': [(0, 0): 0, (109, 111): 13, (111, 106): 13, (107, 108): 103, (111, 112): 4, (24, 23): 10, (72, 71): 18, (81, 83): 13, (63, 62): 9, (5, 4): 26, (0, 0): 0], 'cost': 2074, 'remain_cap': 1}
{'tasks': [(0, 0): 0, (12, 13): 18, (78, 77): 12, (78, 79): 12, (23, 19): 33, (84, 85): 20, (36, 35): 20, (26, 10): 50, (139, 48): 24, (125, 123): 11, (7, 5): 10, (0, 0): 0], 'cost': 2964, 'remain_cap': 0}
{'tasks': [(0, 0): 0, (116, 1): 16, (95, 96): 17, (106, 105): 28, (64, 63): 15, (77, 76): 15, (27, 29): 5, (38, 39): 18, (42, 45): 5, (109, 110): 7, (125, 129): 35, (70, 68): 6, (103, 104): 6, (116, 118): 24, (84, 83): 10, (0, 0): 0], 'cost': 2515, 'remain_cap': 3}
{'tasks': [(0, 0): 0, (66, 68): 4, (10, 7): 48, (26, 27): 6, (96, 97): 3, (66, 67): 9, (44, 43): 14, (8, 7): 37, (10, 11): 25, (105, 104): 15, (11, 12): 12, (107, 106): 25, (106, 109): 8, (54, 55): 4, (0, 0): 0], 'cost': 294

In [ ]:
best

solution cost:6082, feasible
{'tasks': [(0, 0): 0, (106, 105): 28, (104, 103): 6, (96, 97): 3, (96, 95): 17, (95, 94): 8, (54, 139): 14, (48, 47): 10, (24, 23): 10, (24, 26): 19, (26, 27): 6, (27, 28): 5, (33, 44): 4, (72, 71): 18, (71, 70): 9, (68, 66): 4, (66, 65): 33, (1, 116): 16, (0, 0): 0], 'cost': 1134, 'remain_cap': 0}
{'tasks': [(0, 0): 0, (42, 43): 31, (10, 26): 50, (27, 29): 5, (29, 31): 28, (23, 19): 33, (21, 19): 10, (11, 10): 25, (32, 138): 13, (63, 64): 15, (0, 0): 0], 'cost': 1083, 'remain_cap': 0}
{'tasks': [(0, 0): 0, (0, 115): 18, (113, 112): 12, (112, 111): 4, (111, 109): 13, (109, 110): 7, (109, 106): 8, (113, 117): 58, (123, 125): 11, (125, 129): 35, (118, 116): 24, (116, 115): 19, (0, 0): 0], 'cost': 499, 'remain_cap': 1}
{'tasks': [(0, 0): 0, (67, 66): 9, (68, 70): 6, (32, 10): 24, (8, 7): 37, (5, 4): 26, (5, 7): 10, (7, 10): 48, (45, 42): 5, (76, 77): 15, (83, 84): 10, (0, 0): 0], 'cost': 1001, 'remain_cap': 20}
{'tasks': [(0, 0): 0, (78, 77): 12, (72, 43): 36,

In [ ]:
sorted(pop, key=lambda x: x.cost)

[solution cost:6082, feasible
 {'tasks': [(0, 0): 0, (106, 105): 28, (104, 103): 6, (96, 97): 3, (96, 95): 17, (95, 94): 8, (54, 139): 14, (48, 47): 10, (24, 23): 10, (24, 26): 19, (26, 27): 6, (27, 28): 5, (33, 44): 4, (72, 71): 18, (71, 70): 9, (68, 66): 4, (66, 65): 33, (1, 116): 16, (0, 0): 0], 'cost': 1134, 'remain_cap': 0}
 {'tasks': [(0, 0): 0, (42, 43): 31, (10, 26): 50, (27, 29): 5, (29, 31): 28, (23, 19): 33, (21, 19): 10, (11, 10): 25, (32, 138): 13, (63, 64): 15, (0, 0): 0], 'cost': 1083, 'remain_cap': 0}
 {'tasks': [(0, 0): 0, (0, 115): 18, (113, 112): 12, (112, 111): 4, (111, 109): 13, (109, 110): 7, (109, 106): 8, (113, 117): 58, (123, 125): 11, (125, 129): 35, (118, 116): 24, (116, 115): 19, (0, 0): 0], 'cost': 499, 'remain_cap': 1}
 {'tasks': [(0, 0): 0, (67, 66): 9, (68, 70): 6, (32, 10): 24, (8, 7): 37, (5, 4): 26, (5, 7): 10, (7, 10): 48, (45, 42): 5, (76, 77): 15, (83, 84): 10, (0, 0): 0], 'cost': 1001, 'remain_cap': 20}
 {'tasks': [(0, 0): 0, (78, 77): 12, (72, 43

In [ ]:
# QDNS_update_pop(pop, pop[0])